In [1]:
import os
os.environ['CUDA_LAZY_MODULE_LOADING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertModel, BertTokenizer, AdamW
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder
from torch.optim import Adam
from torchviz import make_dot
import shutil
from transformers import  BertConfig
from torch.utils.data import Dataset, DataLoader
import sys
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score
from bs4 import BeautifulSoup
import xml
import numpy as np
from sklearn.model_selection import train_test_split
import re

2023-04-26 22:47:56.804665: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:

print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)


Torch version: 2.0.0+cu118
CUDA available: True
CUDA version: 11.8


In [3]:
ss=df=pd.read_csv('train_set.csv')
test=pd.read_csv('test_data.csv')

In [4]:
X_train,X_test,y = pd.read_csv('X_train.csv'),pd.read_csv('X_test.csv'),pd.read_csv('Y_train.csv')
X_train=X_train.drop('Unnamed: 0',axis=1)
X_test=X_test.drop('Unnamed: 0',axis=1)
y=y.drop('Unnamed: 0',axis=1)
def replace_repeating_newlines(df):
    # Replace all repeating newlines with a single newline in the Text column
    df['Text'] = df['Text'].str.replace('\n+', '\n')

    # Replace all repeating newlines with a single newline in the Titles column
    df['Titles'] = df['Titles'].str.replace('\n+', '\n')

    # Replace all repeating newlines with a single newline in the Links column
    df['Links'] = df['Links'].str.replace('\n+', '\n')

    return df
url_pattern = re.compile(r'https?://[^\s]+.com')
def ret_info(df):
    links=[]
    titles=[]
    text=[]
    for i in range(len(df)):
        h=df.Html.loc[i].strip().lower()
        soup=BeautifulSoup(h,'html.parser')
        script_tags = soup.find_all("script")
        p_tags = soup.find_all("p")
        hreftags= soup.find_all('link')
        script_contents = [script.string for script in script_tags if script.string]
        http_links_in_scripts = [url_pattern.findall(script) for script in script_contents]
        http_links_in_scripts = [url for sublist in http_links_in_scripts for url in sublist]
        https_links = [link["href"] for link in hreftags if link.has_attr("href") and link["href"].startswith("https://")]
        if(http_links_in_scripts==None and https_links==None):
            links.append('')
        links.append(set(http_links_in_scripts+https_links))
        if soup.title!=None:
            titles.append(soup.text.strip())
        else:
            titles.append('')
        text.append([p.text.strip() for p in p_tags])
    for i in range(len(text)):
        text[i]='\n'.join(text[i])
        links[i]='\n'.join(links[i])
    return (links,text,titles)
def find(ass,x):
    for line in ass:
        if line.find(x)!=-1:
            return True
    return False
def Checker(df,links,text,titles):
    a=0
    for i in range(len(links)):
            p=df.Source.loc[i].strip().lower()
            h=df.Html.loc[i].strip().lower()    
            if any(p in link for link in links[i] ):
                a+=1
                continue
            for word in p.split():
                if (titles[i]!=None and titles[i].find(word)!=-1):
                    a+=1
                else:
                    if(p.split()==1):
                        continue
                    for word in p.split():
                        if any(word in link for link in links[i] ):
                            a+=1     
                            break  
                        elif (titles[i]!=None and titles[i].find(word)!=-1):
                            a+=1
                            break  
                        elif (text[i]!=None and find(text[i],word)):
                            a+=1
                            break    
    return a

In [5]:
(links,text,titles) = ret_info(df)
(linkst,textt,titlest) = ret_info(test)
df=pd.DataFrame({'Titles':titles,'Links':links,'Text':text})
fd=pd.DataFrame({'Titles':titlest,'Links':linkst,'Text':textt})
a=0
for i in range(len(df)):
    if (df.Text.loc[i]=='' and df.Links.loc[i]=='' and df.Titles.loc[i]==''):
        df=df.drop(i)
        ss=ss.drop(i)
        a+=1
y=ss.Source

In [6]:
df=replace_repeating_newlines(df)
fd=replace_repeating_newlines(fd)

/tmp/ipykernel_2023/2872246211.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Text'] = df['Text'].str.replace('\n+', '\n')
/tmp/ipykernel_2023/2872246211.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Titles'] = df['Titles'].str.replace('\n+', '\n')
/tmp/ipykernel_2023/2872246211.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Links'] = df['Links'].str.replace('\n+', '\n')


In [7]:
X_train, X_val, y_train, y_val = train_test_split(df, y, test_size=0.1, random_state=42)

In [8]:
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')
max_length=128
batch_size=32

Using cache found in /home/butt_sahab/.cache/torch/hub/huggingface_pytorch-transformers_main


In [9]:
def combine_columns(row):
    return f"Title: {row['Titles']}\nLinks: {row['Links']}\nText: {row['Text']}"

X_train = X_train.apply(combine_columns, axis=1)
X_val = X_val.apply(combine_columns, axis=1)
fd = fd.apply(combine_columns,axis=1)

In [10]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
encoded_categories = label_encoder.fit_transform(y.unique())
labeled_train = label_encoder.transform(y_train)
labeled_val = label_encoder.transform(y_val)

In [11]:
input_ids_train = []
attention_masks_train = []
input_ids_val = []
attention_masks_val = []
input_ids_test = []
attention_masks_test = []

In [12]:
tokenized_data_train =tokenizer(X_train.to_list(), padding=True, truncation=True, max_length=max_length, return_tensors="pt")
tokenized_data_val =tokenizer(X_val.to_list(), padding=True, truncation=True, max_length=max_length, return_tensors="pt")
tokenized_data_test =tokenizer(fd.to_list(), padding=True, truncation=True, max_length=max_length, return_tensors="pt")

In [13]:
input_ids_train.append(tokenized_data_train['input_ids'])
attention_masks_train.append(tokenized_data_train['attention_mask'])
input_ids_val.append(tokenized_data_val['input_ids'])
attention_masks_val.append(tokenized_data_val['attention_mask'])
input_ids_test.append(tokenized_data_test['input_ids'])
attention_masks_test.append(tokenized_data_test['attention_mask'])

In [14]:
input_ids_train = torch.cat(input_ids_train, dim=0)
attention_masks_train = torch.cat(attention_masks_train, dim=0)
input_ids_test = torch.cat(input_ids_test, dim=0)
attention_masks_test= torch.cat(attention_masks_test, dim=0)
input_ids_val = torch.cat(input_ids_val, dim=0)
attention_masks_val= torch.cat(attention_masks_val, dim=0)
labeled_train = torch.tensor(labeled_train)
labeled_val = torch.tensor(labeled_val)

In [15]:
from torch.utils.data import TensorDataset

In [16]:
Dataset_Train = TensorDataset(input_ids_train,attention_masks_train,labeled_train)
Dataset_Test = TensorDataset(input_ids_test,attention_masks_test)
Dataset_Val = TensorDataset(input_ids_val,attention_masks_val,labeled_val)
Train = DataLoader(Dataset_Train, batch_size=batch_size, shuffle=True)
Test = DataLoader(Dataset_Test, batch_size=batch_size, shuffle=False)
Val = DataLoader(Dataset_Val, batch_size=batch_size, shuffle=True)

In [17]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

In [18]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = len(encoded_categories), # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:
import numpy as np
import time
import datetime
import random
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [21]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
total_t0 = time.time()

In [22]:
optimizer=optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

/home/butt_sahab/.local/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [23]:
model.parameters()

<generator object Module.parameters at 0x7f94eb8159e0>

In [24]:
i=0
for param in model.parameters():
    i+=1
    param.requires_grad=True

In [32]:
i=0
for param in model.parameters():
    i+=1
    if(i<20):
        param.requires_grad=False

In [33]:
for param in model.parameters():
        print(param.requires_grad)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
T

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
torch.cuda.empty_cache()
def train(train_dataloader,validation_dataloader, model, optimizer):
    t0 = time.time()
    total_trian_accraucy=0
    total_train_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
        # Progress update every 40 batches.
            # Calculate elapsed time in minutes.
            # Report progress.
           # print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        model.zero_grad()
        output = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        loss=output.loss
        logits=output.logits
        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        total_trian_accraucy += flat_accuracy(logits, label_ids)
        avg_train_accruacy=total_trian_accraucy/len(train_dataloader)
    avg_train_loss = total_train_loss / len(train_dataloader)
    print("")
    print("  Average training loss: {0:.3f}".format(avg_train_loss))
    print("  Average training Accuracy: {0:.3f}".format(avg_train_accruacy))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on

    t0 = time.time()
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        with torch.no_grad():
            output = model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask, 
                                labels=b_labels)
            loss=output.loss
            logits=output.logits
            total_eval_loss += loss.item()
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            total_eval_accuracy += flat_accuracy(logits, label_ids)
            avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)

    print("Validation Accuracy: {0:.2f}".format(avg_val_accuracy))
    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    # Measure how long the validation run took.
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
            

In [28]:
def Validation(test_dataloader, model):
    pred=[]
    for (X,mask) in Test:
        X,mask =X.to(device),mask.to(device)
        with torch.no_grad():
            output = model(X,token_type_ids=None,attention_mask=mask)
            logits=output.logits.detach().cpu().numpy()
            pred.append(np.argmax(logits, axis=1).flatten())
    return pred

In [29]:
print("")
epochs=100
print('Training...')

for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    train(Train,Val,model,optimizer=optimizer)


Training...
======== Epoch 1 / 100 ========

  Average training loss: 6.308
  Average training Accuracy: 0.017
Validation Accuracy: 0.07
  Validation Loss: 6.16
======== Epoch 2 / 100 ========

  Average training loss: 5.890
  Average training Accuracy: 0.137
Validation Accuracy: 0.21
  Validation Loss: 5.79
======== Epoch 3 / 100 ========

  Average training loss: 5.474
  Average training Accuracy: 0.265
Validation Accuracy: 0.32
  Validation Loss: 5.43
======== Epoch 4 / 100 ========

  Average training loss: 5.092
  Average training Accuracy: 0.368
Validation Accuracy: 0.36
  Validation Loss: 5.11
======== Epoch 5 / 100 ========

  Average training loss: 4.760
  Average training Accuracy: 0.424
Validation Accuracy: 0.42
  Validation Loss: 4.85
======== Epoch 6 / 100 ========

  Average training loss: 4.439
  Average training Accuracy: 0.467
Validation Accuracy: 0.43
  Validation Loss: 4.58
======== Epoch 7 / 100 ========

  Average training loss: 4.155
  Average training Accuracy: 

In [31]:
pred=np.concatenate(Validation(Test,model))
Publisher=label_encoder.inverse_transform(pred)
answer=pd.DataFrame({'Id':range(1,len(Publisher)+1),'Publisher':Publisher})
answer=answer.set_index('Id')
answer.to_csv('Answertorch.csv')